In [1]:
import socket
import random
import time
import config_mu4x4
import config_mu5x5
import config_mu6x6
import os
import numpy
from Board import Board
from Board import selffplay



class comp:
    def __init__(self,game):
        self.game=game
        self.turn=-1
        
        
    def load(self):
    
        if self.game == "A4x4":
            self.board = Board(4, 4)

            # Load Model
            checkpoint_folder = r"logs\ataxx_4x4\2024-01-12--14-33-13"
            checkpoint_file = "model.checkpoint"

            # Join the path components
            checkpoint_path = os.path.join(checkpoint_folder, checkpoint_file)


            self.config=config_mu4x4.MuZeroConfig()

        elif self.game=="A5x5":
            self.board = Board(5, 5)

            # Load Model
            checkpoint_folder = r"logs\ataxx_5x5\2024-01-12--09-36-05"
            checkpoint_file = "model.checkpoint"

            # Join the path components    
            checkpoint_path = os.path.join(checkpoint_folder, checkpoint_file)

            self.config=config_mu5x5.MuZeroConfig()

        elif self.game=="A6x6":
            self.board = Board(6, 6)

            # Load Model    
            checkpoint_folder = r"logs\ataxx_6x6\2024-01-11--19-39-14"
            checkpoint_file = "model.checkpoint"

            # Join the path components    
            checkpoint_path = os.path.join(checkpoint_folder, checkpoint_file)


            self.config=config_mu6x6.MuZeroConfig()


        

        
        self.model=self.board.load_model(checkpoint_path)
        self.selfplay=selffplay(self.model,numpy.random.randint(10000),self.board,self.config)


    def jogo(self,ag):
        if(ag==1):
            self.turn=1
        else:
            self.turn=-1
                
        action=self.selfplay.play_game(0,0,self.turn)
        x,y,x2,y2=self.board.step(action,self.turn)
     
        
        return f"MOVE {x},{y},{x2},{y2}"
    
    def receive(self,move,ag):
        if(ag==1):
            self.turn=-1
        else:
            self.turn=1
            
        
        x=int(move[-7])
        y=int(move[-5])
        x2=int(move[-3])
        y2=int(move[-1])
        
        distx,disty=self.board.GetDistanceBoardUnits((x,y),(x2,y2))
        if (distx == 1 and disty <= 1) or (disty == 1 and distx <= 1):
                    
                self.board.MakeNewPieceAt((x2,y2), self.turn)
                self.board.CatchPiece((x2,y2), self.turn)
                       
                
        elif (distx == 2 and disty <= 2) or (disty == 2 and distx <= 2):
                    
                self.board.MovePieceTo((x,y), (x2,y2), self.turn)
                self.board.CatchPiece((x2,y2), self.turn)
                 
      
        run = self.board.CheckGame()      
        return run


pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Game="A4x4" # "A6x6" "G7x7" "G9x9" "A5x5"


def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    
    CO=comp(Game) # inicialize the game
    CO.load()  # load the correspondent model
    
    if "1" in response:
        ag=1
    else:
        ag=2
        
    first=True
    run=True
    
    while run:
        
        
        if ag == 1 or not first:
            
            move = CO.jogo(ag)  # select the best move to execute
            time.sleep(1)
            client_socket.send(str(move).encode())
            print("Send:",move)
        
            # Wait for server response
            response = client_socket.recv(1024).decode()
            
            print(f"Server Response1: {response}")
            if "END" in response: break
         
        first=False
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")
        run=CO.receive(response,ag)  # verify if the game continues
        
            
        if "END" in response: break


   
    client_socket.close()

if __name__ == "__main__":
    connect_to_server()

Server ResponseINIT: AG1 A6x6
Playing: A6x6
Send: MOVE 5,0,4,1
Server Response1: VALID
Server Response2: MOVE 5,5,4,3
Send: MOVE 4,1,4,2
Server Response1: VALID
Server Response2: MOVE 0,0,1,0
Send: MOVE 4,1,5,3
Server Response1: VALID
Server Response2: MOVE 1,0,3,2
Send: MOVE 5,3,3,5
Server Response1: VALID
Server Response2: MOVE 0,0,1,2
Send: MOVE 5,0,4,0
Server Response1: VALID
Server Response2: MOVE 4,3,4,5
Send: MOVE 4,0,2,2
Server Response1: VALID
Server Response2: MOVE 3,5,3,3
Send: MOVE 0,5,2,3
Server Response1: VALID
Server Response2: MOVE 4,2,4,3
Send: MOVE 1,2,0,3
Server Response1: VALID
Server Response2: MOVE 3,2,2,4
Send: MOVE 0,3,1,3
Server Response1: VALID
Server Response2: MOVE 3,3,4,1
Send: MOVE 2,2,4,0
Server Response1: VALID
Server Response2: MOVE 4,2,2,2
Send: MOVE 4,0,5,1
Server Response1: VALID
Server Response2: MOVE 1,2,2,0
Send: MOVE 0,3,0,4
Server Response1: VALID
Server Response2: MOVE 2,0,0,1
Send: MOVE 2,4,4,4
Server Response1: VALID
Server Response2: MOVE 2,

ValueError: invalid literal for int() with base 10: ' '